# CFPB Consumer Complaints
Analysis of CFPB's public [Consumer Complaints Database] (http://www.consumerfinance.gov/data-research/consumer-complaints/)

**Visual Analysis**
* Complaints by company, issue, product, and tags
* Complaints over time
* Complaints by geographic region (state, zip code)

**NLP**
* Topic modeling on consumer narrative
* Topic modeling on company response

## Data Setup
---
* import statements
* read in data from Socrata API

In [1]:
#import statements
import numpy as np
import pandas as pd
import datetime
import urllib

# Visualization
from pandas.tools.plotting import scatter_matrix
import seaborn as sns
sns.set(style="ticks", color_codes=True)
import matplotlib.pyplot as plt
% matplotlib inline
plt.style.use('fivethirtyeight')

# plotly setup (Visualization)
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True)

In [2]:
# Access Complaints data via Socrata API
# source for connection code: https://dev.socrata.com/foundry/data.consumerfinance.gov/jhzv-w97w
query = ("https://data.consumerfinance.gov/resource/jhzv-w97w.json")
raw_data = pd.read_json(query)

In [3]:
# confirm data load
print raw_data.shape
raw_data.head()

(632107, 18)


,company,company_public_response,company_response,complaint_id,complaint_what_happened,consumer_consent_provided,consumer_disputed,date_received,date_sent_to_company,issue,product,state,sub_issue,sub_product,submitted_via,tags,timely,zip_code
0,Bank of America,Company chooses not to provide a public response,Closed with explanation,1780421,NaN,N/A,No,2016-02-09T16:58:05.000,2016-02-09T19:52:53.000,Problems caused by my funds being low,Bank account or service,NY,NaN,Checking account,Referral,NaN,Yes,11201
1,Synchrony Financial,NaN,Closed with explanation,1543488,XX/XX/XXXX I was approached by XXXX Attorney X...,Consent provided,No,2015-08-29T18:08:50.000,2015-09-04T14:55:10.000,Other fee,Credit card,TX,NaN,NaN,Web,NaN,Yes,787XX
2,Ocwen,NaN,Closed with explanation,1820018,I have been struggling with my mortgage paymen...,Consent provided,No,2016-03-08T01:29:34.000,2016-03-08T01:29:35.000,"Loan modification,collection,foreclosure",Mortgage,CA,NaN,Conventional fixed mortgage,Web,Older American,Yes,946XX
3,"SunTrust Banks, Inc.",NaN,Closed with explanation,629839,NaN,N/A,No,2013-12-11T17:05:57.000,2013-12-13T05:00:00.000,Cont'd attempts collect debt not owed,Debt collection,FL,Debt is not mine,I do not know,Phone,NaN,Yes,33056
4,"Select Portfolio Servicing, Inc",Company believes it acted appropriately as aut...,Closed with explanation,1774742,NaN,N/A,No,2016-02-05T19:36:58.000,2016-02-09T20:29:51.000,Other,Mortgage,CA,NaN,Other mortgage,Referral,NaN,Yes,92551


In [4]:
# check columns and data types
raw_data.dtypes

company                      object
company_public_response      object
company_response             object
complaint_id                  int64
complaint_what_happened      object
consumer_consent_provided    object
consumer_disputed            object
date_received                object
date_sent_to_company         object
issue                        object
product                      object
state                        object
sub_issue                    object
sub_product                  object
submitted_via                object
tags                         object
timely                       object
zip_code                     object
dtype: object

## Data Wrangling
---
* Date / time columns
* Aggregations by company, product, issue, state
* Calculate averages
* Normalize complaint counts by company

In [8]:
# make a copy of raw data for wrangling
complaints = raw_data.copy()

In [10]:
# recast datetime columns
complaints['date_received'] = pd.to_datetime(complaints['date_received'], infer_datetime_format=True)
complaints['date_sent_to_company'] = pd.to_datetime(complaints['date_sent_to_company'], infer_datetime_format=True)

# confirm results
print complaints.dtypes
complaints.head()

company                              object
company_public_response              object
company_response                     object
complaint_id                          int64
complaint_what_happened              object
consumer_consent_provided            object
consumer_disputed                    object
date_received                datetime64[ns]
date_sent_to_company         datetime64[ns]
issue                                object
product                              object
state                                object
sub_issue                            object
sub_product                          object
submitted_via                        object
tags                                 object
timely                               object
zip_code                             object
dtype: object


,company,company_public_response,company_response,complaint_id,complaint_what_happened,consumer_consent_provided,consumer_disputed,date_received,date_sent_to_company,issue,product,state,sub_issue,sub_product,submitted_via,tags,timely,zip_code
0,Bank of America,Company chooses not to provide a public response,Closed with explanation,1780421,NaN,N/A,No,2016-02-09 16:58:05,2016-02-09 19:52:53,Problems caused by my funds being low,Bank account or service,NY,NaN,Checking account,Referral,NaN,Yes,11201
1,Synchrony Financial,NaN,Closed with explanation,1543488,XX/XX/XXXX I was approached by XXXX Attorney X...,Consent provided,No,2015-08-29 18:08:50,2015-09-04 14:55:10,Other fee,Credit card,TX,NaN,NaN,Web,NaN,Yes,787XX
2,Ocwen,NaN,Closed with explanation,1820018,I have been struggling with my mortgage paymen...,Consent provided,No,2016-03-08 01:29:34,2016-03-08 01:29:35,"Loan modification,collection,foreclosure",Mortgage,CA,NaN,Conventional fixed mortgage,Web,Older American,Yes,946XX
3,"SunTrust Banks, Inc.",NaN,Closed with explanation,629839,NaN,N/A,No,2013-12-11 17:05:57,2013-12-13 05:00:00,Cont'd attempts collect debt not owed,Debt collection,FL,Debt is not mine,I do not know,Phone,NaN,Yes,33056
4,"Select Portfolio Servicing, Inc",Company believes it acted appropriately as aut...,Closed with explanation,1774742,NaN,N/A,No,2016-02-05 19:36:58,2016-02-09 20:29:51,Other,Mortgage,CA,NaN,Other mortgage,Referral,NaN,Yes,92551


In [ ]:
# Aggregate by Company
company = complaints[['company', 'complaint_id']].groupby('company').agg({'complaint_id': pd.Series.nunique}).reset_index()

# rename columns
company = company.rename(columns={'complaint_id':'total_complaints'})

# sort by descending complaint counts
company.sort_values(by='total_complaints', ascending=False, inplace=True)

# review results
company.head()

In [ ]:
# Aggregate by State
state = complaints[['state', 'complaint_id']].groupby('state').agg({'complaint_id': pd.Series.nunique}).reset_index()

# rename columns
state = state.rename(columns={'complaint_id':'total_complaints'})

# sort by descending complaint counts
state.sort_values(by='total_complaints', ascending=False, inplace=True)

# review results
state.head()

In [ ]:
# Aggregate by product
product = complaints[['product', 'complaint_id']].groupby('product').agg({'complaint_id': pd.Series.nunique}).reset_index()

# rename columns
product = product.rename(columns={'complaint_id':'total_complaints'})

# sort by descending complaint counts
product.sort_values(by='total_complaints', ascending=False, inplace=True)

# review results
product.head()

In [ ]:
# Aggregate by issue
issue = complaints[['issue', 'complaint_id']].groupby('issue').agg({'complaint_id': pd.Series.nunique}).reset_index()

# rename columns
issue = issue.rename(columns={'complaint_id':'total_complaints'})

# sort by descending complaint counts
issue.sort_values(by='total_complaints', ascending=False, inplace=True)

# review results
issue.head()

In [ ]:
# Aggregate by tags
tags = complaints[['tags', 'complaint_id']].groupby('tags').agg({'complaint_id': pd.Series.nunique}).reset_index()

# rename columns
tags = tags.rename(columns={'complaint_id':'total_complaints'})

# sort by descending complaint counts
tags.sort_values(by='total_complaints', ascending=False, inplace=True)

# review results
tags.head()

In [ ]:
# average complaints totals
avg_complaints_company = np.mean(company.total_complaints)
print 'Average complaints by company:', avg_complaints_company

avg_complaints_state = np.mean(state.total_complaints)
print 'Average complaints by state:', avg_complaints_state

avg_complaints_product = np.mean(product.total_complaints)
print 'Average complaints by product:', avg_complaints_product

avg_complaints_issue = np.mean(issue.total_complaints)
print 'Average complaints by issue:', avg_complaints_issue

avg_complaints_tags = np.mean(tags.total_complaints)
print 'Average complaints by tags:', avg_complaints_tags

In [ ]:
# Calculate average yearly complaints by company


In [ ]:
# normalize complaints by yearly average to adjust for bigger companies that receive more complaints


In [ ]:
# issues by company response published or not

# issues by response type

# commonly disputed issues

## Data Blending
---
* Census
* Zipcode to latitude and longitude
* Company type and size
* Military base locations
* Bank branch locations

*Note: Location analysis is impacted by scrubbed dataset. States are provided for all complaints, but some complaints only include zipcodes aggregated to 3 digit level.*

## Visual Analysis
---
Using pandas visuals, seaborn, and plotly
* Complaints by company, product, state over time
* Outliers by issue grouped by company

In [ ]:
trace1 = go.Scatter(
    y = company[''],
    mode='markers',
    marker=dict(
        size='16',
        color = company['product'], #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
)
data = [trace1]

iplot(data, show_link=False)

In [ ]:
iplot({
    'data': [
        Scatter(x=df[continent+'_Life Expentancy [in years]'],
                y=df[continent+'_Gross Domestic Product per Capita [in USD of the year 2000]'],
                text=df[continent+'_text'],
                marker=Marker(size=df[continent+'_marker.size'], sizemode='area', sizeref=131868,),
                mode='markers',
                name=continent) for continent in ['Africa', 'Americas', 'Asia', 'Europe', 'Oceania']
    ],
    'layout': Layout(xaxis=XAxis(title='Life Expectancy'), yaxis=YAxis(title='GDP per Capita', type='log'))
}, show_link=False)

## Regression Analysis
---
* Correlation
* Linear Regression
* Random Forest

## Topic Modeling
---
* By Consumer Narrative
* By Company Response